In [1]:
# TODO: move to utils

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

def get_data(split=True):
    
    data = load_breast_cancer()

    X = data.data
    y = data.target
    
    if split:
        return train_test_split(X, y, test_size=0.25, random_state=42)
    else:
        return X, y

# Dane

In [2]:
import numpy as np

X_train, X_test, y_train, y_test = get_data()

## Zadanie 1.1

Używając modelu [`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) z pakietu sklearn uzyskać 100% dokładność (mierzoną miarą [`accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html))na zbiorze treningowym. Państwa zadanie polega na dobraniu paramtru `gamma`, dla ułatwienia proszę nie zmieniać pozotsałych domyślnych paramertów. Zalecany przedział parametru `gamma` to wartości z przedziału [0, 1] w skali logarytmicznej.

In [3]:
from sklearn.svm import SVC

# your code here

auto_gamma = 1 / X_train.shape[1]
scale_gamma = 1 / (X_train.shape[1] * X_train.var())

best_train_acc = 0

for g in [auto_gamma, scale_gamma, 0.00001, 0.0001, 0.001, 0.01, 0.1]:

    svm = SVC(C=1, gamma=g)

    svm.fit(X_train, y_train)
    
    train_score = svm.score(X_train, y_train)
    
    if train_score > best_train_acc:
        best_gamma = g
        best_train_acc = train_score
    
    print(f"Gamma: {g:0.6f}", end='\t')
    print(f"train: {train_score:0.3f}")

Gamma: 0.033333	train: 1.000
Gamma: 0.000001	train: 0.911
Gamma: 0.000010	train: 0.915
Gamma: 0.000100	train: 0.941
Gamma: 0.001000	train: 0.977
Gamma: 0.010000	train: 1.000
Gamma: 0.100000	train: 1.000


In [4]:
# test
best_gamma = best_gamma # ???

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
train_acc = svm.score(X_train, y_train)

assert train_acc == 1.

## Zadanie 1.2
Używając tej samej rodziny modeli znajdź tym razem taką wartośc `gamma`, która daje najlepszy wynik na zbiorze **testowym**. Powinieneś(-aś) być w stanie osiągnąć wynik conajmniej `0.95` accuracy. 

In [5]:
from sklearn.svm import SVC

auto_gamma = 1 / X_train.shape[1]
scale_gamma = 1 / (X_train.shape[1] * X_train.var())

best_test_acc = 0

for g in [auto_gamma, scale_gamma, 0.00001, 0.0001, 0.001, 0.01, 0.1]:

    svm = SVC(gamma=g)

    svm.fit(X_train, y_train)
    
    train_score = svm.score(X_train, y_train)
    
    print(f"Gamma: {g:0.6f}", end='\t')
    print(f"train: {train_score:0.3f}", end=' ')
    
    test_score = svm.score(X_test, y_test)
    
    if test_score > best_test_acc:
        best_test_acc = test_score
        best_gamma = g
    
    print(f"test: {test_score:0.3f}")

Gamma: 0.033333	train: 1.000 test: 0.622
Gamma: 0.000001	train: 0.911 test: 0.951
Gamma: 0.000010	train: 0.915 test: 0.951
Gamma: 0.000100	train: 0.941 test: 0.965
Gamma: 0.001000	train: 0.977 test: 0.923
Gamma: 0.010000	train: 1.000 test: 0.629
Gamma: 0.100000	train: 1.000 test: 0.622


In [6]:
# test
best_gamma = best_gamma # ???

svm = SVC(gamma=best_gamma)
svm.fit(X_train, y_train)
test_acc = svm.score(X_test, y_test)

assert test_acc >= 0.95

### Problem.

**W poprzednim zadaniu zakładaliśmy, że podział na zbiór trenujący/testujący jest nam podany z góry, ale co jeśli go nie mamy?**

Możemy oczywiście sami arbitralnie wybrać część datasetu i uznać ją za nasz zbiór testowy, ale to mogą się z tym wiązać dodatkowe problemy: co jeśli wybrany przez nas fragment jest akurat różny od reszty datasetu, lub odwrotnie?

**Rozwiązanie:** Walidacja Krzyżowa.

1. Podziel dataset na zadaną przez parametr `k` liczbę (prawie) równych grup.
2. Dla każdego podziału, zwróć jedną z tych części jako zbiór testowy, a sumę reszty jako zbiów treningowy.
3. Po nauczeniu łącznie `k` modeli, uśrednij ich wyniki na zbiorach testowych i zwróć jako ostateczny wynik.

## Zadanie 2.

Państwa zadaniem jest zaimplementowanie walidacji krzyżowej, czyli funkcji, która dla podanego datasetu w postaci macierzy danych `X` i wektora etykiet `y` zwróci listę `k` krotek: 
  
  `(treningowe_dane, treningowe_etykiety, testowe_dane, testowe_etykiety)`
  
podziałów dokonanych przez walidacje krzyżową. Następnie należy użyć modelu z poprzedniego zadania dla policzenia dokładności na zbiorze testowym dla walidacji krzyżowej.

Proszę **nie** korzystać z gotowych rozwiązań dostępnych w pakiecie sklearn.


In [7]:
from typing import List, Tuple


def cross_validation(X: np.ndarray, y: np.ndarray, k: int) -> List[Tuple[np.ndarray, np.ndarray, 
                                                                         np.ndarray, np.ndarray]]:
    
    splits = []
    
    N = X.shape[0]
    split_size = N // k
    
    all_ids = set(range(N))
    
    for i, start_id in enumerate(range(0, N, split_size)):
        if (i + 1) < k:
            test_ids = set(range(start_id, start_id + split_size))
        elif (i + 1) == k:
            test_ids = set(range(start_id, N))
        else:
            break
        
        train_ids = list(all_ids - test_ids)
        test_ids = list(test_ids)
        
        X_tr = X[train_ids]
        y_tr = y[train_ids]
        
        X_te = X[test_ids]
        y_te = y[test_ids]
        
        splits.append((X_tr, y_tr, X_te, y_te))
    
    return splits


In [12]:
test_cv(cross_validation)

In [9]:
X, y = get_data(split=False)

# your code here

accs = []

splits = cross_validation(X, y, k=3)

for X_train, y_train, X_test, y_test in splits:
    
    svm = SVC(gamma=0.01)
    svm.fit(X_train, y_train)
    test_acc = svm.score(X_test, y_test)
    
    accs.append(test_acc)
    
cv_accuracy = np.mean(accs) # ???

In [11]:
# TODO move to checker

def test_cv(cv_func):
    
    X, y = get_data(split=False)
    k = 5
    
    splits = cv_func(X=X, y=y, k=k)
    
    assert len(splits) == k, "Wrong number of splits returned!"
    
    for split in splits:
        assert len(split) == 4, f"Wrong tuple length for a single split, expected 4, got: {len(split)}"
        assert isinstance(split[0], np.ndarray), "Wrong type returned!"
        assert isinstance(split[1], np.ndarray), "Wrong type returned!"
        assert isinstance(split[2], np.ndarray), "Wrong type returned!"
        assert isinstance(split[3], np.ndarray), "Wrong type returned!"
        
        assert len(split[0]) == len(split[1]), f"Train split shape mismatch: {len(split[0])} and {len(split[1])}"
        assert len(split[2]) == len(split[3]), f"Test split shape mismatch: {len(split[0])} and {len(split[1])}"
        
    assert sum([split[2].shape[0] for split in splits]) == X.shape[0], \
        "Sum of all split lengths is mismatched with whole dataset, did you use whole dataset for splitting?"
                                                                                                     
                                                                            

## Zadanie 3

Mając już lepszą metode walidacji naszego rozwiązania Państwa zadaniem jest znalezienia najlepszego zestawu hiperparametrów dla modelu z poprzednich zadań, lecz tym razem będziemy szukać również parametru `C`. Parametru C zaleca się szukać w przedziale $(0, + \infty)$ również w skali logarytmicznej.

W zadaniu należy oczywiście skorzystać z zaimplementowanej przez Państwa walidacji krzyżowej. Ponieważ dla dwóch parametrów `C` oraz `gamma` możliwych kombinacji do przetestowania robi są dość sporo dla przetestowania dużych zakresów zalecane są również inne metody przeszukiwania, takie jak:

* przeszukiwanie po kolei z jednym z parametów ustalonym na stałą.
* przeszukiwanie losowe obu parametrów

Oczywiście jeśli zasoby pozlową można szukać tzw. "grid searchem".

Powinno udać się Państwu wyciągnąć przynamniej `0.94` accuracy na walidacji krzyżowej.

In [13]:
# grid search

from sklearn.svm import SVC

X, y = get_data(split=False)

auto_gamma = 1 / X_train.shape[1]
scale_gamma = 1 / (X_train.shape[1] * X_train.var())

results = {}

for c in np.logspace(-3, 3, 7):
    for g in [auto_gamma, scale_gamma, 0.00001, 0.0001, 0.001, 0.01, 0.1]:

        accs = []

        splits = cross_validation(X, y, k=3)

        for X_train, y_train, X_test, y_test in splits:

            svm = SVC(C=c, gamma=g)
            svm.fit(X_train, y_train)
            test_acc = svm.score(X_test, y_test)

            accs.append(test_acc)

        cv_accuracy = np.mean(accs) 

        results[(c, g)] = cv_accuracy
        
best_params, best_score = sorted(results.items(), key=lambda x: x[1], reverse=True)[0]
print(best_params)
print(best_score)

(1000.0, 5.856208850389724e-07)
0.9455109559821602


In [14]:
# random search

X, y = get_data(split=False)

results = {}

max_iter = 50

for i in range(max_iter):
    
    c = np.random.choice(np.logspace(-10, 10, 21))
    g = np.random.choice(np.logspace(-10, -1, 11))

    accs = []

    splits = cross_validation(X, y, k=3)

    for X_train, y_train, X_test, y_test in splits:

        svm = SVC(C=c, gamma=g)
        svm.fit(X_train, y_train)
        test_acc = svm.score(X_test, y_test)

        accs.append(test_acc)

    cv_accuracy = np.mean(accs) 

    results[(c, g)] = cv_accuracy
    
best_params, best_score = sorted(results.items(), key=lambda x: x[1], reverse=True)[0]
print(best_params)
print(best_score)

(10000000.0, 5.011872336272725e-08)
0.9490198251105756


In [15]:
# one-by-one search

X, y = get_data(split=False)

from sklearn.svm import SVC

auto_gamma = 1 / X_train.shape[1]
scale_gamma = 1 / (X_train.shape[1] * X_train.var())


best_acc = 0.
    
for c in np.logspace(-10, 10, 21):
    
    svm = SVC(C=c, gamma='scale')

    svm.fit(X_train, y_train)

    acc = svm.score(X_test, y_test)

    if acc > best_acc:
        best_c = c
        best_acc = acc

best_acc = 0.
    
for g in np.logspace(-10, -1, 11):
    
    svm = SVC(C=best_c, gamma=g)

    svm.fit(X_train, y_train)

    acc = svm.score(X_test, y_test)

    if acc > best_acc:
        best_gamma = g
        best_acc = acc

print(best_c, best_gamma)
print(best_acc)

100000.0 3.981071705534969e-07
0.9476439790575916


## Zadanie 4.

Załóżmy, że naszym problmem jest zdecydowanie, która rodzina modeli *SVM* najlepiej radzi sobei z naszym datasetem. Przez rodzinę rozumiemy tutaj modele SVM z różną *funkcją bazwoą* (zwaną często *funkcją jądra*). W pakiecie mamy dostępne kilka możliwości, włącznie z podawaniem swoich własnych, ale w tym zadaniu skupimy się na czterach dostępnych od ręki: `linear`, `poly`, `rbf`, `sigmoid`.

Wiemy jak znaleźć najlepszy zestaw parametrów dla danej rodziny modeli, zrobiliśmy to do tej pory dla domyślnej funkcji bazowej czyli `rbf`. Jak jednak mamy "uczciwie" porównać wyniki modeli pomiędzy sobą? Do tej pory patrzyliśmy na wyniki modelu dla datasetu testowego i to na podstawie tego wyniku wybieraliśmy najlepsze hiperparametry. Jakie mogą być z tym problemy? Overfitting?

Rozwiązanie: jeszcze jedna walidacja krzyżowa!

1. Pierwsza walidacja krzyżowna podzieli nam nasz zbiór na treningowy i testowy. Te testowe zbiory będą naszymi ostatecznymi zbiorami testowymi, na których nie będziemy w żaden sposób się uczyć czy szukać hiperparametrów. 
2. Następnie nasz zbiór treningowy podzielimy ponownie walidacją krzyżową na dwie części: faktyczny treningowy i walidacyjny. Tych dwóch podziałów będziemy używać jak poprzednio do uczenia modelu i testowania hiperparametrów.
3. Po znalezieniu najlepszego zestawu hiperparametrów nauczymy ostatecznie nasz model na sumie zbiorów treningowego i walidacyjnego i sprawdzimy jego dokładność na ostatecznym zbiorze testowym.


**Uwaga**: paramter `C` używany jest dla każdej możliwej funkcji bazowej. Proszę sprawdzić jakie parametry są używane dla jakch funkcji jądra. 
**Hint**: parametry, które mogą państwa interesować to oczywiście `kernel`, oraz `C`, `degree`, `gamma`, `coef0`.

In [17]:
from collections import defaultdict

X, y = get_data(split=False)

cs = np.logspace(-3, 3, 7)
degrees = [2, 3, 4]
gammas = np.logspace(-4, -1, 5)
coefs = [-1, 0, 1]

k_cv = 3

kernels_with_grids = {'linear': {'C': cs}, 
                      'poly': {'C': cs,
                               'degree': degrees,
                               'gamma': gammas,
                               'coef0': coefs},
                      'rbf': {'C': cs,
                              'gamma': gammas},
                      'sigmoid': {'C': cs,
                               'gamma': gammas,
                               'coef0': coefs}}

def get_random_params(n_iter, params):
    for i in range(n_iter):
        yield {name: np.random.choice(param_space) for name, param_space in params.items()}

results = {}

for kernel, grid in kernels_with_grids.items():
    
    kernal_res = []
    kernel_accs = []
    
    splits = cross_validation(X, y, k=k_cv)
    
    for i, (X_train_valid, y_train_valid, X_test, y_test) in enumerate(splits):
        
        for param_set in get_random_params(n_iter=3, params=grid):
            accs = []
            
            inner_split = cross_validation(X_train_valid, y_train_valid, k=k_cv)
            
            for X_train, y_train, X_valid, y_valid in inner_split:
                
                svm = SVC(kernel=kernel, **param_set)
                svm.fit(X_train, y_train)
                acc = svm.score(X_valid, y_valid)
                
                accs.append(acc)
            
            kernal_res.append((param_set, np.mean(accs)))
            
        best_set, best_acc = sorted(kernal_res, key=lambda x: x[1], reverse=True)[0]
        
        best_svm = SVC(kernel=kernel, **best_set)
        best_svm.fit(X_train_valid, y_train_valid)
        kernel_score = best_svm.score(X_test, y_test)
        
        kernel_accs.append(kernel_score)
        print(f"Done: {kernel} {i+1}/{len(splits)}")
        
    results[kernel] = np.mean(kernel_accs)
    
    
results

Done: linear 1/3
Done: linear 2/3
Done: linear 3/3
Done: poly 1/3
Done: poly 2/3
Done: poly 3/3
Done: rbf 1/3
Done: rbf 2/3
Done: rbf 3/3
Done: sigmoid 1/3
Done: sigmoid 2/3
Done: sigmoid 3/3


{'linear': 0.9455109559821602,
 'poly': 0.9472746244124952,
 'rbf': 0.7535850485239665,
 'sigmoid': 0.6269148729881714}